## Load pretrained model on QM9 and predict 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
from scannet.models import SCANNet
from utils.general import process_xyz, prepare_input

In [ ]:
# model path for QM9 property
model_path = '../pretrained_models/qm9/homo/model.h5'
model_load = SCANNet.load_model_infer(model_path)

### Predict from xyz files

In [ ]:
name = 'Thymine'
struct = process_xyz('../experiments/molecules/{}.xyz'.format(name))
# Computing Neighbor from Voronoi tessalation with cutoff threshold
ip = prepare_input(struct, use_ring=True,
                   w_t=0.2, d_t=4.0)

In [ ]:
energy, ga_scores = model_load.predict(ip)
print(energy)

In [ ]:
ga_scores = np.squeeze(ga_scores)
# Showing atom and corresponding global attention score for property
for i in range(len(struct['Atoms'])):
    print(struct['Atoms'][i], ga_scores[i])

In [ ]:
# Export prediction result to xyz file for Visualization using Ovito
with open('visualize/{}_homo.xyz'.format(name), 'w') as f:
    f.write(str(len(struct['Atoms'])) + '\n')
    f.write('XXX \n')
    for i in range(len(struct['Atoms'])):
        f.write('{}\t{}\t{}\t{}\t{}\n'.format(struct['Atoms'][i], struct['Coords'][i][0],
            struct['Coords'][i][1], struct['Coords'][i][2],
            ga_scores[i]))